In [1]:
import pandas as pd
import requests
import nltk
import re
from bs4 import BeautifulSoup
import numpy as np
from gensim.models import Word2Vec
import pandas as pd
import re
import nltk
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from collections import Counter 
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
lst_stopwords = nltk.corpus.stopwords.words('russian')
lst_stopwords.extend(['…', '«', '»', '...'])

import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
df=pd.read_excel("../data/Список_сайтов_для_определения_тематики (6).xlsx")
df

,Ссылки,Категория,Тема
0,http://school04.ru/,Образование,Школа и вуз
1,https://www.dns-shop.ru/catalog/17a8df6816404e...,Электроника,Принтеры и МФУ
2,https://sbermegamarket.ru/catalog/audiosistemy...,Электроника,"Фото, видео и аудиотехника"
3,http://do.vanmk.ru/index.php/distantsionnoe-ob...,Образование,Дополнительное образование и курсы
4,https://www.eldorado.ru/d/tekhnika-dlya-kukhni/,Бытовая техника,Бытовая техника
...,...,...,...
300,https://uslugi.yandex.ru/213-moscow/category/r...,"Строительство, обустройство и ремонт",Ремонт
301,https://biletix.ru/,Отдых и путешествия,Билеты
302,https://students.kpfu.ru/node/14072,Красота и здоровье,Здоровое питание
303,https://www.coursera.org/browse,Образование,Дополнительное образование и курсы


In [ ]:
# df[['Категория', 'Тема']] = df[['Категория', 'Тема']].apply (lambda x: pd.factorize(x)[0])

df['Категория_code'] = pd.factorize(df['Категория'])[0]
df['Тема_code'] = pd.factorize(df['Тема'])[0]

# Создайте словарь, где ключи - это коды, а значения - соответствующие текстовые значения
col1 = dict(zip(df['Категория_code'], df['Категория']))
col2 = dict(zip(df['Тема_code'], df['Тема']))

# Выведите словари
print("Словарь для Колонка1:")
print(col1)

print("\nСловарь для Колонка2:")
print(col2)

Словарь для Колонка1:
{0: 'Образование', 1: 'Электроника', 2: 'Бытовая техника', 3: 'Отдых и путешествия', 4: 'Животные', 5: 'Еда и напитки', 6: 'Бизнес', 7: 'Семья и дети', 8: 'Недоступно', 9: 'Развлечения и досуг', 10: 'Финансы', 11: 'Недвижимость', 12: 'Строительство, обустройство и ремонт', 13: 'Спорт', 14: 'Красота и здоровье', 15: 'Канцелярские товары', 16: 'Одежда, обувь и аксессуары', 17: 'Транспорт', 18: 'Телеком', 19: 'Сельскохозяйственное оборудование и техника', 20: 'Подарки и цветы'}

Словарь для Колонка2:
{0: 'Школа и вуз', 1: 'Принтеры и МФУ', 2: 'Фото, видео и аудиотехника', 3: 'Дополнительное образование и курсы', 4: 'Бытовая техника', 5: 'Билеты', 6: 'Зарубежный туризм', 7: 'Российский туризм', 8: 'Домашние животные', 9: 'Доставка воды', 10: 'Юридические услуи', 11: 'Свадьба', 12: 'Недоступно', 13: 'Прокат автомобилей', 14: 'Игры', 15: 'Книги', 16: 'Товары для детей', 17: 'Банки и кредиты', 18: 'Покупка и аренда', 19: 'Дизайн интерьера', 20: 'Компьютерная техника', 21

In [ ]:
# def extract_text_from_url(url):
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')
#             text = ' '.join([tag.get_text() for tag in soup.find_all()])
#             # Удалите символы \n
#             text = text.replace('\n', ' ')
#             return text
#         else:
#             return None
#     except Exception as e:
#         print(f"Произошла ошибка при запросе к {url}: {str(e)}")
#         return None
# df['HTML'] = df['Ссылки'].apply(extract_text_from_url)

In [ ]:
def get_html(row):
    try:
        response = requests.get(row['Ссылки'])
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_text = [tag.get('content') for tag in meta_tags if tag.get('name') == 'description']
            if meta_text==[] or meta_text=='':
                meta_text=None
            return ', '.join(meta_text)
        else:
            return None
    except Exception as e:
        return None



# Примените функцию к каждой строке DataFrame и создайте новый столбец
df['HTML'] = df.apply(get_html, axis=1)

In [ ]:
df['HTML'].replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df

,Ссылки,Категория,Тема,Категория_code,Тема_code,HTML
8,https://pet-mir.ru/,Животные,Домашние животные,4,8,В мире домашних животных
9,https://otzivi-tut.ru/catalog/kompanii/kompani...,Еда и напитки,Доставка воды,5,9,⭐⭐⭐⭐⭐ Проверенные отзывы покупателей и клиенто...
10,https://ok.ru/consultantplus.comp,Бизнес,Юридические услуи,6,10,Группа КонсультантПлюс - компания в Одноклассн...
13,https://www.youtube.com/watch?v=ogpew7zuu9y,Недоступно,Недоступно,8,12,"Смотрите любимые видео, слушайте любимые песни..."
16,https://landofgames.ru/,Развлечения и досуг,Игры,9,14,Игровой портал Land of Games включает в себя н...
...,...,...,...,...,...,...
300,https://uslugi.yandex.ru/213-moscow/category/r...,"Строительство, обустройство и ремонт",Ремонт,12,35,"Ремонт квартир и домов в Москве, 31514 отзывов..."
301,https://biletix.ru/,Отдых и путешествия,Билеты,3,5,Мы помогаем находить самые дешевые авиабилеты....
302,https://students.kpfu.ru/node/14072,Красота и здоровье,Здоровое питание,14,28,Основные принципы формирования здорового образ...
303,https://www.coursera.org/browse,Образование,Дополнительное образование и курсы,0,3,Choose from hundreds of free courses or pay to...


In [ ]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, stem=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   characters and then strip)
    text = str(text).lower()  
    text = re.sub(r"\[(.*?)\]", "", text)  
    text = re.sub(r"\s+", " ", text)  
    text = re.sub(r"\w+…|…", "", text)  
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    lst_text = [t for t in lst_text if not t in lst_stopwords]  
    lst_text = ["" if t.isdigit() else t for t in lst_text]  
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
    
    if stem == True:
        stemmer = nltk.SnowballStemmer(language="russian")
        lst_text = [stemmer.stem(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)

    return text.split(" ")

In [ ]:
df["HTML_CL"] = df["HTML"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=True, flg_lemm=True,stem=True,
          lst_stopwords=lst_stopwords))

In [ ]:
df

,Ссылки,Категория,Тема,Категория_code,Тема_code,HTML,HTML_CL,vectors
8,https://pet-mir.ru/,Животные,Домашние животные,4,8,В мире домашних животных,"[мир, домашн, животн]","[-0.0019343247, -0.00077248487, -0.00017137865..."
9,https://otzivi-tut.ru/catalog/kompanii/kompani...,Еда и напитки,Доставка воды,5,9,⭐⭐⭐⭐⭐ Проверенные отзывы покупателей и клиенто...,"[⭐⭐⭐⭐⭐, проверен, отзыв, покупател, клиент, ко...","[-0.00054474734, -5.579805e-05, -0.0004060603,..."
10,https://ok.ru/consultantplus.comp,Бизнес,Юридические услуи,6,10,Группа КонсультантПлюс - компания в Одноклассн...,"[групп, консультантплюс, компан, одноклассник,...","[0.0008769262, 0.00037059892, -8.655344e-05, 0..."
13,https://www.youtube.com/watch?v=ogpew7zuu9y,Недоступно,Недоступно,8,12,"Смотрите любимые видео, слушайте любимые песни...","[смотр, любим, виде, слуша, любим, песн, загру...","[0.00029977536, 0.0007749513, -0.00045863274, ..."
16,https://landofgames.ru/,Развлечения и досуг,Игры,9,14,Игровой портал Land of Games включает в себя н...,"[игров, порта, land, of, game, включа, новост,...","[-0.00026030323, 0.0011907616, -0.00015144687,..."
...,...,...,...,...,...,...,...,...
300,https://uslugi.yandex.ru/213-moscow/category/r...,"Строительство, обустройство и ремонт",Ремонт,12,35,"Ремонт квартир и домов в Москве, 31514 отзывов...","[ремонт, квартир, дом, москв, , отзыв, средн, ...","[0.00026431543, 0.00083516166, 0.00079581176, ..."
301,https://biletix.ru/,Отдых и путешествия,Билеты,3,5,Мы помогаем находить самые дешевые авиабилеты....,"[помога, наход, сам, дешев, авиабилет, biletix...","[-0.00043357507, 0.0005983438, -8.1452774e-05,..."
302,https://students.kpfu.ru/node/14072,Красота и здоровье,Здоровое питание,14,28,Основные принципы формирования здорового образ...,"[основн, принцип, формирован, здоров, образ, ж...","[-0.0004636588, 3.4316337e-05, -0.0002895467, ..."
303,https://www.coursera.org/browse,Образование,Дополнительное образование и курсы,0,3,Choose from hundreds of free courses or pay to...,"[choos, from, hundr, of, free, cours, or, pay,...","[-1.0364185e-05, 0.0004523106, 0.0008194735, 0..."


In [ ]:
def vectorize(list_of_docs, model):
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features

In [ ]:
model_w2v = Word2Vec(sentences=df['HTML_CL'].values, vector_size=300, window=5, min_count=1, workers=4)

In [ ]:
vectorized_docs= vectorize(df['HTML_CL'], model=model_w2v)
len(vectorized_docs), len(vectorized_docs[0])

(162, 300)

In [ ]:
df['vectors']=vectorized_docs
df

,Ссылки,Категория,Тема,Категория_code,Тема_code,HTML,HTML_CL,vectors
8,https://pet-mir.ru/,Животные,Домашние животные,4,8,В мире домашних животных,"[мир, домашн, животн]","[0.0004568358, 0.0014246536, 5.0366198e-05, -0..."
9,https://otzivi-tut.ru/catalog/kompanii/kompani...,Еда и напитки,Доставка воды,5,9,⭐⭐⭐⭐⭐ Проверенные отзывы покупателей и клиенто...,"[⭐⭐⭐⭐⭐, проверен, отзыв, покупател, клиент, ко...","[-0.0005532297, 0.0004083504, -0.00010952264, ..."
10,https://ok.ru/consultantplus.comp,Бизнес,Юридические услуи,6,10,Группа КонсультантПлюс - компания в Одноклассн...,"[групп, консультантплюс, компан, одноклассник,...","[0.00061100273, 0.0009408241, -6.1712494e-06, ..."
13,https://www.youtube.com/watch?v=ogpew7zuu9y,Недоступно,Недоступно,8,12,"Смотрите любимые видео, слушайте любимые песни...","[смотр, любим, виде, слуша, любим, песн, загру...","[-0.00011535541, 0.00066408445, -0.00063081464..."
16,https://landofgames.ru/,Развлечения и досуг,Игры,9,14,Игровой портал Land of Games включает в себя н...,"[игров, порта, land, of, game, включа, новост,...","[5.4466676e-05, 0.0013439179, 1.6256561e-05, -..."
...,...,...,...,...,...,...,...,...
300,https://uslugi.yandex.ru/213-moscow/category/r...,"Строительство, обустройство и ремонт",Ремонт,12,35,"Ремонт квартир и домов в Москве, 31514 отзывов...","[ремонт, квартир, дом, москв, , отзыв, средн, ...","[-0.0007664202, 0.00027191592, 0.00015547764, ..."
301,https://biletix.ru/,Отдых и путешествия,Билеты,3,5,Мы помогаем находить самые дешевые авиабилеты....,"[помога, наход, сам, дешев, авиабилет, biletix...","[0.00038813168, -0.0005689325, 0.00084043114, ..."
302,https://students.kpfu.ru/node/14072,Красота и здоровье,Здоровое питание,14,28,Основные принципы формирования здорового образ...,"[основн, принцип, формирован, здоров, образ, ж...","[-0.00040938638, -9.419119e-05, -0.00013956624..."
303,https://www.coursera.org/browse,Образование,Дополнительное образование и курсы,0,3,Choose from hundreds of free courses or pay to...,"[choos, from, hundr, of, free, cours, or, pay,...","[0.00026786886, 0.00043768526, 0.00013993228, ..."


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(df['vectors'].values.tolist()).astype('float32'), np.asarray(df[['Категория_code',"Тема_code"]].values.tolist()).astype('float'), test_size = 0.33, random_state = 42)

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

model = keras.Sequential([
    layers.Input(shape=(300, 1)),  # Входной слой размерности (300, 1) для одномерных данных
    layers.Conv1D(128, 5, activation='relu'),  # Слой свертки с 128 фильтрами и функцией активации ReLU
    layers.MaxPooling1D(2),  # Слой максимальной пулинга
    layers.Conv1D(64, 5, activation='relu'),  # Еще один слой свертки с 64 фильтрами и функцией активации ReLU
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 5, activation='relu'),  # Еще один слой свертки с 64 фильтрами и функцией активации ReLU
    layers.MaxPooling1D(2),  # Еще один слой максимальной пулинга
    layers.Flatten(),  # Преобразуем выход сверточных слоев в плоский вектор
    layers.Dense(2, activation='linear')  # Выходной слой с 2 нейронами и функцией активации linear (для регрессии)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Подготовка данных, если x_test и y_test имеют размерность (batch_size, 300)
x_test = x_test.reshape((-1, 300, 1))

# Обучаем модель на данных
model.fit(x_test, y_test, epochs=700, batch_size=32)

Epoch 1/700
2/2 [==============================] - 1s 23ms/step - loss: 369.5725 - mae: 14.7634
Epoch 2/700
2/2 [==============================] - 0s 23ms/step - loss: 367.9123 - mae: 14.7180
Epoch 3/700
2/2 [==============================] - 0s 21ms/step - loss: 363.9590 - mae: 14.5953
Epoch 4/700
2/2 [==============================] - 0s 22ms/step - loss: 355.4594 - mae: 14.3429
Epoch 5/700
2/2 [==============================] - 0s 28ms/step - loss: 339.0051 - mae: 13.8250
Epoch 6/700
2/2 [==============================] - 0s 27ms/step - loss: 313.6136 - mae: 13.0414
Epoch 7/700
2/2 [==============================] - 0s 20ms/step - loss: 273.6274 - mae: 11.7534
Epoch 8/700
2/2 [==============================] - 0s 20ms/step - loss: 223.6684 - mae: 10.0779
Epoch 9/700
2/2 [==============================] - 0s 20ms/step - loss: 170.9132 - mae: 8.9878
Epoch 10/700
2/2 [==============================] - 0s 22ms/step - loss: 134.2073 - mae: 9.2296
Epoch 11/700
2/2 [=======================

In [ ]:
df.iloc[1]

Ссылки            https://otzivi-tut.ru/catalog/kompanii/kompani...
Категория                                             Еда и напитки
Тема                                                  Доставка воды
Категория_code                                                    5
Тема_code                                                         9
HTML              ⭐⭐⭐⭐⭐ Проверенные отзывы покупателей и клиенто...
HTML_CL           [⭐⭐⭐⭐⭐, проверен, отзыв, покупател, клиент, ко...
vectors           [-0.0005532297, 0.0004083504, -0.00010952264, ...
Name: 9, dtype: object

In [ ]:
predicted_output = model.predict(df['vectors'][28].reshape(1, -1))
predicted_output

1/1 [==============================] - 0s 112ms/step


array([[ 8.294934, 22.069172]], dtype=float32)

In [ ]:

text=['''Блог о здоровом питании''']
text_prep=pd.DataFrame(text,columns=['text'])
text_prep
text_prep['clean']=text_prep['text'].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=True, flg_lemm=True, 
          lst_stopwords=lst_stopwords))
vec_text=vectorize(text_prep['text'], model=model_w2v)
vec_text
pr = model.predict(np.asarray(vec_text).reshape(1, -1))
pr


1/1 [==============================] - 0s 28ms/step


array([[ 8.407902, 22.349524]], dtype=float32)

In [ ]:
result = df.loc[df['Категория'] == 6 | (df['Тема'] <=20)]
result

TypeError: '<=' not supported between instances of 'str' and 'int'

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

from sklearn.multioutput import MultiOutputClassifier

# Замените RandomForestClassifier на ваш классификатор
base_classifier = RandomForestClassifier()

# Оберните его в MultiOutputClassifier
classifier = MultiOutputClassifier(base_classifier)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', classifier)
])

# Определите параметры для поиска по сетке
param_grid = {
    'classifier__estimator__n_estimators': [50, 100, 200],
    'classifier__estimator__max_depth': [None, 10, 20],
    # Добавьте другие параметры вашего классификатора
}

# Создайте объект GridSearchCV и продолжайте как раньше
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(x_train, y_train)

# Выведите лучшие параметры
print("Лучшие параметры:")
print(grid_search.best_params_)

# Получите лучшую модель
best_model = grid_search.best_estimator_

# Сделайте предсказания на тестовой выборке
y_pred = best_model.predict(x_test)

# Выведите метрики классификации
print("Метрики классификации:")
# print(classification_report(y_test, y_pred))

# Выведите точность
# accuracy = accuracy_score(y_test, y_pred)
# print("Точность:", accuracy)

# Сохраните лучшую модель (по желанию)
import joblib
joblib.dump(best_model, 'best_model.pkl')

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Лучшие параметры:
{'classifier__estimator__max_depth': 10, 'classifier__estimator__n_estimators': 200}


ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [ ]:
grid_search.predict(vec_text)

array([[ 8., 12.]])